In [1]:
import os

os.environ['HF_HOME'] = '/home/ubuntu/scicom'

In [2]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="Scicom-intl/sort-multilingual-tts",
    repo_type='dataset', local_dir='./'
)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 441 files: 100%|██████████| 441/441 [00:00<00:00, 2839.54it/s]


'/home/ubuntu/scicom'

In [3]:
from streaming import MDSWriter
from streaming.base.format.mds.encodings import Encoding, _encodings
from streaming import LocalDataset
from glob import glob
import numpy as np
import json
import pandas as pd
from tqdm import tqdm
from multiprocess import Pool
import itertools

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)

def multiprocessing(strings, function, cores=6, returned=True):
    df_split = chunks(strings, len(strings) // cores)
    pool = Pool(cores)
    pooled = pool.map(function, df_split)
    pool.close()
    pool.join()

    if returned:
        return list(itertools.chain(*pooled))

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
class UInt32(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint32)

_encodings['uint32'] = UInt32

columns = {
    'input_ids': 'uint32',
    'position_ids': 'uint32',
    'attention_mask': 'uint32',
    'audio': 'str',
    'text': 'str'
}
hashes = 'sha1', 'xxh64'

In [7]:
# df = pd.read_parquet('sort-merge.parquet')
# df

In [6]:
with open('bin-packing.json') as fopen:
    bin_packing = json.load(fopen)

len(bin_packing)

614552

In [9]:
!rm -rf multipacking
!mkdir multipacking

In [10]:
def collator(batch, batch_position_ids):
    input_ids = []
    position_ids = []
    masks = []
    for i in range(len(batch)):
        l = len(batch[i])
        input_ids.extend(batch[i])
        position_ids.extend(batch_position_ids[i])
        masks.append(l)
    
    return {
        'input_ids': np.array(input_ids).astype(np.uint32),
        'position_ids': np.array(position_ids).astype(np.uint32),
        'attention_mask': np.array(masks).astype(np.uint32),
        'audio': '',
        'text': '',
    }

def loop(indices):
    indices, index = indices
    out_root = f'multipacking/tokenized-{index}'
    os.system(f'rm -rf {out_root}')
    df = pd.read_parquet('sort-merge.parquet')
    datasets = {}
    for f in glob('tokenized-4k-qwen3/*'):
        dataset = LocalDataset(local=f)
        datasets[f] = dataset

    with MDSWriter(out=out_root, columns=columns, compression=None, hashes=hashes) as out:
        for packing in tqdm(indices):
            rows = df.iloc[packing]
            temp = []
            position_ids = []
            for k in range(rows.shape[0]):
                input_ids = datasets[rows.iloc[k]['f']][rows.iloc[k]['i']]['input_ids']
                temp.append(input_ids)
                position_ids.append(range(len(input_ids)))
                
            o = collator(temp, position_ids)
            out.write(o)

In [13]:
# loop((bin_packing[:100], 0))

In [ ]:
multiprocessing(bin_packing, loop, cores = 30, returned = False)

 95%|█████████▍| 19454/20485 [01:42<00:07, 142.88it/s]

In [ ]:
folders = sorted(glob('multipacking/tokenized-*'), key = lambda x: int(x.split('-')[-1]))
folders

In [ ]:
!rm -rf multipacking-final

In [ ]:
with MDSWriter(out='multipacking-final', columns=columns, compression=None, hashes=hashes) as out:
    for f in folders:
        try:
            dataset = LocalDataset(local=f)
            for i in tqdm(range(len(dataset))):
                out.write(dataset[i])
        except Exception as e:
            print(e)
            pass